## 4. 数据操作

In [52]:
import numpy as np
import torch

### 4.1 numpy和torch构建特殊向量是一样的

In [53]:
(np.arange(3), torch.arange(3))

(array([0, 1, 2]), tensor([0, 1, 2]))

### 4.2 torch和numpy对一维向量的reshape操作也是一样

In [54]:
(np.arange(6).reshape(2,3), torch.arange(6).reshape(2,3))

(array([[0, 1, 2],
        [3, 4, 5]]),
 tensor([[0, 1, 2],
         [3, 4, 5]]))

### 4.3 torch和numpy构建特殊矩阵也是一样的

In [57]:
(np.ones((2,3)), torch.zeros((2,3)))

(array([[1., 1., 1.],
        [1., 1., 1.]]),
 tensor([[0., 0., 0.],
         [0., 0., 0.]]))

### 4.4 torch和numpy向量的长度属性和方法不一致

In [59]:
(np.arange(3).shape, torch.arange(3).shape,
np.arange(3).size, torch.arange(3).numel())

((3,), torch.Size([3]), 3, 3)

### 4.5 torch和numpy的数据类型转换不一样

In [61]:
(np.arange(3).astype(np.int16),
torch.arange(3).to(torch.float64))

(array([0, 1, 2], dtype=int16), tensor([0., 1., 2.], dtype=torch.float64))

### 4.6 torch和numpy 指数操作

In [66]:
(np.exp(np.arange(6, dtype=np.float32).reshape(2,-1)),
 torch.exp(torch.arange(6, dtype=torch.float32).reshape(3,-1)))

(array([[  1.       ,   2.718282 ,   7.3890557],
        [ 20.085537 ,  54.59815  , 148.41316  ]], dtype=float32),
 tensor([[  1.0000,   2.7183],
         [  7.3891,  20.0855],
         [ 54.5981, 148.4132]]))

In [48]:
torch.exp(y.reshape(-1,).to(torch.int16))

tensor([  2.7183,   7.3891,  20.0855,  20.0855,  54.5981, 148.4132])

### 4.7 torch和numpy在向量，矩阵的合并操作上不一样

##### 1. torch.cat函数有dim参数，代表连接方向
##### 2. torch.cat函数接收list or tuple
##### 3. numpy.c_[]和numpy.r_[]函数 接收 分开传入的向量

In [73]:
np.c_[np.ones(3).reshape(-1,1), np.zeros(3).reshape(-1,1)], \
np.r_[np.ones(3), np.zeros(3)], \
torch.cat([torch.ones(3).reshape(-1,1), torch.zeros(3).reshape(-1,1)], dim=1), \
torch.cat((torch.ones(3), torch.zeros(3)), dim=0)

(array([[1., 0.],
        [1., 0.],
        [1., 0.]]),
 array([1., 1., 1., 0., 0., 0.]),
 tensor([[1., 0.],
         [1., 0.],
         [1., 0.]]),
 tensor([1., 1., 1., 0., 0., 0.]))

### 4.8 tensor的matrix如果是单索引，则选择行。多索引 -> 用:表示选择这一维度的所有

In [77]:
x = np.arange(4).reshape(2,-1)
y = torch.arange(4).reshape(2,-1)
x[0], y[0], x[1,:], y[1,:]

(array([0, 1]), tensor([0, 1]), array([2, 3]), tensor([2, 3]))

### 4.9 python中的原位操作

In [87]:
x = np.arange(6).reshape(2,-1)
first_id = id(x)
x[:] = 2
second_id = id(x)
x += 3
third_id = id(x)
x = 2
fourth_id = id(x)
first_id, second_id, third_id, fourth_id

(3078336861488, 3078336861488, 3078336861488, 140710099149640)

#### 4.10 numpy和torch之间的转换

In [94]:
torch.tensor(np.arange(3)), torch.ones(2).tolist()

(tensor([0, 1, 2], dtype=torch.int32), [1.0, 1.0])

#### 4.11 如何利用with，file.write，一行一行的写入数据

In [107]:
import os
f_path = './test.csv'
with open(f_path, 'w') as f:
    f.write('con,enz,res\n')
    f.write('1,p,23\n')
    f.write('2,a,NA\n')
    f.write('3,NA,NA\n')

#### 4.12 利用pandas读取文件

In [129]:
import pandas as pd
df = pd.read_csv('./test.csv')
df

,con,enz,res
0,1,p,23.0
1,2,a,NaN
2,3,NaN,NaN


#### 4.13 在pandas中，利用iloc，提取predictors和outcome

In [112]:
predictor, outcome = df.iloc[:, :-1], df.iloc[:, -1]
predictor, outcome

(   con  enz
 0    1    p
 1    2    a
 2    3  NaN,
 0    23.0
 1     NaN
 2     NaN
 Name: res, dtype: float64)

#### 4.14 在pandas中，对整个df的所有numeric列填充NA

In [119]:
df.select_dtypes('number').fillna(df.select_dtypes('number').mean())

,con,res
0,1,23.0
1,2,23.0
2,3,23.0


#### 4.15 在pandas中，对qualitative列做dummy操作，并且忽视numeric列

In [121]:
pd.get_dummies(df, dummy_na=True)

,con,res,enz_a,enz_p,enz_nan
0,1,23.0,0,1,0
1,2,NaN,1,0,0
2,3,NaN,0,0,1


#### 4.16 在pandas对数据做完预处理后，用torch.tensor()将DataFrame转变成tensor

In [130]:
df = pd.get_dummies(df, dummy_na=True)
df_number = df.select_dtypes(include=np.number)
df_object = df.select_dtypes(include=np.object_)
df_number = df_number.fillna(df_number.mean())
df = pd.concat([df_number, df_object], axis=1)
df

,con,res,enz_a,enz_p,enz_nan
0,1,23.0,0,1,0
1,2,23.0,1,0,0
2,3,23.0,0,0,1


In [146]:
torch.tensor(df.loc[:, ['con', 'enz_a', 'enz_p', 'enz_nan']].to_numpy()), \
torch.tensor(df.loc[:, 'res'])

(tensor([[1, 0, 1, 0],
         [2, 1, 0, 0],
         [3, 0, 0, 1]]),
 tensor([23., 23., 23.], dtype=torch.float64))